In [35]:
import itertools
import gurobipy as gpy
import numpy as np

n = 10
locs = range(n)
d = np.genfromtxt(f'mst_data/dist_{n}.txt')

''' Construct Gurobi model and optimize '''
m = gpy.Model('mst')
x = [m.addVar(vtype=gpy.GRB.BINARY,name='x_'+str(i)+'_'+str(j)) for i in range(n) for j in range(i+1,n)]
m.update()
m.setObjective(gpy.quicksum(x[i] * d[i] for i in range(len(x))), gpy.GRB.MINIMIZE)  
m.addLConstr(gpy.quicksum(x), gpy.GRB.EQUAL, rhs=n-1)
m.update()
m.optimize()

models = [gpy.Model("Null"), m.copy()]
i = 0

while models[i].NumConstrs != models[i+1].NumConstrs:
    ''' Search for subtours '''
    ''' Consider all subsets of locations for all possible number of subset sizes m '''
    for mm in range(2,n):
        ''' Construct all m-location combinations of the location set '''
        subsets = itertools.combinations(locs,mm)
        ''' construct constraints for all of the combinations '''
        for subset in subsets:
            num_links = sum([m.getVarByName(f'x_{i}_{j}').x for i,j in itertools.combinations(subset,2)])
            if num_links >= len(subset):
                ''' Construct subtour constraint for this combination '''
                m.addConstr(gpy.quicksum(m.getVarByName(f'x_{i}_{j}') for i,j in itertools.combinations(subset,2)) <= len(subset) - 1)
        
    ''' The model needs to be updated after all constraints are added, otherwise 
            the previous solution is deleted '''
    m.update()  
    m.optimize()
    models.append(m.copy())
    i += 1


''' Print solution '''
for v in m.getVars():
    print(f'{v.varName}: {v.x}')

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i9-12900H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 1 rows, 45 columns and 45 nonzeros
Model fingerprint: 0x0cce2455
Variable types: 0 continuous, 45 integer (45 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+00, 9e+00]
Found heuristic solution: objective 415.0000000
Presolve removed 1 rows and 45 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 2: 234 415 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.340000000000e+02, best bound 2.340000000000e+02, gap 0.0000%
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CP